In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import *

from sklearn.preprocessing import PowerTransformer


import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [24]:
train = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [25]:
test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")
test.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
0,1200000,28.0,Female,2310.0,NaN,4.0,Bachelor's,Self-Employed,7.657981,Rural,Basic,NaN,19.0,NaN,1.0,2023-06-04 15:21:39.245086,Poor,Yes,Weekly,House
1,1200001,31.0,Female,126031.0,Married,2.0,Master's,Self-Employed,13.381379,Suburban,Premium,NaN,14.0,372.0,8.0,2024-04-22 15:21:39.224915,Good,Yes,Rarely,Apartment
2,1200002,47.0,Female,17092.0,Divorced,0.0,PhD,Unemployed,24.354527,Urban,Comprehensive,NaN,16.0,819.0,9.0,2023-04-05 15:21:39.134960,Average,Yes,Monthly,Condo
3,1200003,28.0,Female,30424.0,Divorced,3.0,PhD,Self-Employed,5.136225,Suburban,Comprehensive,1.0,3.0,770.0,5.0,2023-10-25 15:21:39.134960,Poor,Yes,Daily,House
4,1200004,24.0,Male,10863.0,Divorced,2.0,High School,Unemployed,11.844155,Suburban,Premium,NaN,14.0,755.0,7.0,2021-11-26 15:21:39.259788,Average,No,Weekly,House


In [26]:
print(f"Train Shape : {train.shape}")
print(f"Test Shape : {test.shape}")


Train Shape : (1200000, 21)
Test Shape : (800000, 20)


# Combining Test and Train data for processing

In [27]:
df = pd.concat([train, test])

In [28]:
df.shape

(2000000, 21)

# Setting all columns to appropriate datatypes

In [29]:
df[["Previous Claims", "Number of Dependents", "Vehicle Age", "Age", "Insurance Duration"]] = df[["Previous Claims", "Number of Dependents", "Vehicle Age", "Age", "Insurance Duration"]].astype("object")

In [30]:
df["Policy Start Date"] = pd.to_datetime(df["Policy Start Date"])

In [31]:
nulls = []
nuniques = []
uniques = []
types = []

for i in df.columns:
    nulls.append(df[i].isnull().sum())
    nuniques.append(df[i].nunique())
    uniques.append(df[i].unique())
    types.append(df[i].dtype)

In [32]:
pd.DataFrame(
    {
        "Column" : df.columns,
        "Data Type" : types,
        "Nulls" : nulls,
        "No. of Uniques" : nuniques,
        "Uniques" : uniques
    }
).sort_values(by="Nulls", ascending=False)

,Column,Data Type,Nulls,No. of Uniques,Uniques
20,Premium Amount,float64,800000,4794,"[2869.0, 1483.0, 567.0, 765.0, 2022.0, 3202.0,..."
11,Previous Claims,object,606831,10,"[2.0, 1.0, 0.0, nan, 3.0, 4.0, 5.0, 6.0, 7.0, ..."
7,Occupation,object,597200,3,"[Self-Employed, nan, Employed, Unemployed]"
13,Credit Score,float64,229333,550,"[372.0, 694.0, nan, 367.0, 598.0, 614.0, 807.0..."
5,Number of Dependents,object,182802,5,"[1.0, 3.0, 2.0, 0.0, 4.0, nan]"
16,Customer Feedback,object,130100,3,"[Poor, Average, Good, nan]"
8,Health Score,float64,123525,811360,"[22.59876067181393, 15.569730989408043, 47.177..."
3,Annual Income,float64,74809,97540,"[10049.0, 31678.0, 25602.0, 141855.0, 39651.0,..."
1,Age,object,31194,47,"[19.0, 39.0, 23.0, 21.0, 29.0, 41.0, 48.0, 44...."
4,Marital Status,object,30865,3,"[Married, Divorced, Single, nan]"


# EDA - Filling Nulls

In [33]:
new_train = df.iloc[:1200000, :]

In [34]:
df.drop(columns=["id"], inplace=True)
new_train.drop(columns=["id"], inplace=True)

In [42]:
def return_splits(ddf, feature_name, target_name):
    return [ddf[ddf[feature_name] == i][target_name] for i in ddf[feature_name].unique()]

In [67]:
def give_stats_analysis(df, target_column_name):
    ddf = df.copy()
    ddf = ddf.dropna()

    features = []
    tests = []
    stats = []
    pvals = []
    count = 0

    target = ddf[target_column_name]
    for i in ddf.columns:
        features.append(i)
        feature = ddf[i]
        
        if (feature.dtype == "O" and (target.dtype == "float" or target.dtype == "int")) or (target.dtype == "O" and (feature.dtype == "float" or feature.dtype == "int")):
            stat, pval, *_ = kruskal(*return_splits(ddf, feature.name, target.name))
            tests.append("Kruskal-Wallis")
            stats.append(stat)
            pvals.append(pval)
            
        
        elif (feature.dtype == "float" or feature.dtype == "int") and (target.dtype == "float" or target.dtype == "int"):
            stat, pval, *_ = spearmanr(feature, target)
            tests.append("SpearmanR")
            stats.append(stat)
            pvals.append(pval)

        elif feature.dtype == "O" and target.dtype == "O":
            stat, pval, *_ = chi2_contingency(pd.crosstab(feature, target))
            tests.append("Chi-Square")
            stats.append(stat)
            pvals.append(pval)
        
        else:
            tests.append(np.nan)
            stats.append(np.nan)
            pvals.append(np.nan)

        print(f"{feature.name} ■■■ {target_column_name}".ljust(50, "-")+"✅")
    
    return pd.DataFrame({
        "Feature" : features,
        "Target" : [target_column_name]*ddf.shape[1],
        "Statistic Test" : tests,
        "Test Statistic" : stats,
        "P-Value" : pvals
    }).sort_values(by="P-Value")

In [69]:
give_stats_analysis(new_train, "Premium Amount")

Age ■■■ Premium Amount----------------------------✅
Gender ■■■ Premium Amount-------------------------✅
Annual Income ■■■ Premium Amount------------------✅
Marital Status ■■■ Premium Amount-----------------✅
Number of Dependents ■■■ Premium Amount-----------✅
Education Level ■■■ Premium Amount----------------✅
Occupation ■■■ Premium Amount---------------------✅
Health Score ■■■ Premium Amount-------------------✅
Location ■■■ Premium Amount-----------------------✅
Policy Type ■■■ Premium Amount--------------------✅
Previous Claims ■■■ Premium Amount----------------✅
Vehicle Age ■■■ Premium Amount--------------------✅
Credit Score ■■■ Premium Amount-------------------✅
Insurance Duration ■■■ Premium Amount-------------✅
Policy Start Date ■■■ Premium Amount--------------✅
Customer Feedback ■■■ Premium Amount--------------✅
Smoking Status ■■■ Premium Amount-----------------✅
Exercise Frequency ■■■ Premium Amount-------------✅
Property Type ■■■ Premium Amount------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
19,Premium Amount,Premium Amount,SpearmanR,1.000000,0.000000e+00
2,Annual Income,Premium Amount,SpearmanR,-0.063926,0.000000e+00
10,Previous Claims,Premium Amount,Kruskal-Wallis,949.478664,1.194784e-199
12,Credit Score,Premium Amount,SpearmanR,-0.040824,2.580178e-141
7,Health Score,Premium Amount,SpearmanR,0.013625,3.073783e-17
0,Age,Premium Amount,Kruskal-Wallis,80.549007,1.227264e-03
6,Occupation,Premium Amount,Kruskal-Wallis,6.775390,3.378647e-02
15,Customer Feedback,Premium Amount,Kruskal-Wallis,4.388035,1.114680e-01
5,Education Level,Premium Amount,Kruskal-Wallis,5.752873,1.242724e-01
9,Policy Type,Premium Amount,Kruskal-Wallis,3.388502,1.837368e-01


In [71]:
give_stats_analysis(new_train, "Previous Claims")

Age ■■■ Previous Claims---------------------------✅
Gender ■■■ Previous Claims------------------------✅
Annual Income ■■■ Previous Claims-----------------✅
Marital Status ■■■ Previous Claims----------------✅
Number of Dependents ■■■ Previous Claims----------✅
Education Level ■■■ Previous Claims---------------✅
Occupation ■■■ Previous Claims--------------------✅
Health Score ■■■ Previous Claims------------------✅
Location ■■■ Previous Claims----------------------✅
Policy Type ■■■ Previous Claims-------------------✅
Previous Claims ■■■ Previous Claims---------------✅
Vehicle Age ■■■ Previous Claims-------------------✅
Credit Score ■■■ Previous Claims------------------✅
Insurance Duration ■■■ Previous Claims------------✅
Policy Start Date ■■■ Previous Claims-------------✅
Customer Feedback ■■■ Previous Claims-------------✅
Smoking Status ■■■ Previous Claims----------------✅
Exercise Frequency ■■■ Previous Claims------------✅
Property Type ■■■ Previous Claims-----------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
10,Previous Claims,Previous Claims,Chi-Square,3.072032e+06,0.000000e+00
2,Annual Income,Previous Claims,Kruskal-Wallis,7.732378e+04,4.211139e-218
7,Health Score,Previous Claims,Kruskal-Wallis,2.503823e+05,1.315595e-123
12,Credit Score,Previous Claims,Kruskal-Wallis,1.357447e+03,3.796840e-70
19,Premium Amount,Previous Claims,Kruskal-Wallis,6.339186e+03,1.283284e-62
4,Number of Dependents,Previous Claims,Chi-Square,8.562323e+01,8.786966e-07
5,Education Level,Previous Claims,Chi-Square,4.408025e+01,7.467726e-03
18,Property Type,Previous Claims,Chi-Square,2.678293e+01,4.395978e-02
6,Occupation,Previous Claims,Chi-Square,2.608544e+01,5.283779e-02
11,Vehicle Age,Previous Claims,Chi-Square,1.791785e+02,6.523835e-02


In [72]:
give_stats_analysis(new_train, "Occupation")

Age ■■■ Occupation--------------------------------✅
Gender ■■■ Occupation-----------------------------✅
Annual Income ■■■ Occupation----------------------✅
Marital Status ■■■ Occupation---------------------✅
Number of Dependents ■■■ Occupation---------------✅
Education Level ■■■ Occupation--------------------✅
Occupation ■■■ Occupation-------------------------✅
Health Score ■■■ Occupation-----------------------✅
Location ■■■ Occupation---------------------------✅
Policy Type ■■■ Occupation------------------------✅
Previous Claims ■■■ Occupation--------------------✅
Vehicle Age ■■■ Occupation------------------------✅
Credit Score ■■■ Occupation-----------------------✅
Insurance Duration ■■■ Occupation-----------------✅
Policy Start Date ■■■ Occupation------------------✅
Customer Feedback ■■■ Occupation------------------✅
Smoking Status ■■■ Occupation---------------------✅
Exercise Frequency ■■■ Occupation-----------------✅
Property Type ■■■ Occupation----------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
6,Occupation,Occupation,Chi-Square,768008.000000,0.000000e+00
7,Health Score,Occupation,Kruskal-Wallis,246379.238601,1.111416e-72
2,Annual Income,Occupation,Kruskal-Wallis,68893.978816,3.537980e-23
19,Premium Amount,Occupation,Kruskal-Wallis,5085.719378,5.080758e-08
18,Property Type,Occupation,Chi-Square,29.377775,6.550726e-06
3,Marital Status,Occupation,Chi-Square,21.387437,2.652954e-04
16,Smoking Status,Occupation,Chi-Square,8.878429,1.180521e-02
12,Credit Score,Occupation,Kruskal-Wallis,624.035555,1.430868e-02
8,Location,Occupation,Chi-Square,11.152711,2.490032e-02
10,Previous Claims,Occupation,Chi-Square,26.085444,5.283779e-02


In [73]:
give_stats_analysis(new_train, "Credit Score")

Age ■■■ Credit Score------------------------------✅
Gender ■■■ Credit Score---------------------------✅
Annual Income ■■■ Credit Score--------------------✅
Marital Status ■■■ Credit Score-------------------✅
Number of Dependents ■■■ Credit Score-------------✅
Education Level ■■■ Credit Score------------------✅
Occupation ■■■ Credit Score-----------------------✅
Health Score ■■■ Credit Score---------------------✅
Location ■■■ Credit Score-------------------------✅
Policy Type ■■■ Credit Score----------------------✅
Previous Claims ■■■ Credit Score------------------✅
Vehicle Age ■■■ Credit Score----------------------✅
Credit Score ■■■ Credit Score---------------------✅
Insurance Duration ■■■ Credit Score---------------✅
Policy Start Date ■■■ Credit Score----------------✅
Customer Feedback ■■■ Credit Score----------------✅
Smoking Status ■■■ Credit Score-------------------✅
Exercise Frequency ■■■ Credit Score---------------✅
Property Type ■■■ Credit Score--------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
12,Credit Score,Credit Score,SpearmanR,1.000000,0.000000e+00
2,Annual Income,Credit Score,SpearmanR,-0.150058,0.000000e+00
10,Previous Claims,Credit Score,Kruskal-Wallis,902.866264,1.360192e-189
19,Premium Amount,Credit Score,SpearmanR,-0.040824,2.580178e-141
7,Health Score,Credit Score,SpearmanR,0.005636,4.782809e-04
5,Education Level,Credit Score,Kruskal-Wallis,8.701385,3.353623e-02
11,Vehicle Age,Credit Score,Kruskal-Wallis,28.715509,7.059371e-02
0,Age,Credit Score,Kruskal-Wallis,60.062079,7.976144e-02
4,Number of Dependents,Credit Score,Kruskal-Wallis,7.936039,9.394948e-02
15,Customer Feedback,Credit Score,Kruskal-Wallis,3.715746,1.560041e-01


In [74]:
give_stats_analysis(new_train, "Number of Dependents")

Age ■■■ Number of Dependents----------------------✅
Gender ■■■ Number of Dependents-------------------✅
Annual Income ■■■ Number of Dependents------------✅
Marital Status ■■■ Number of Dependents-----------✅
Number of Dependents ■■■ Number of Dependents-----✅
Education Level ■■■ Number of Dependents----------✅
Occupation ■■■ Number of Dependents---------------✅
Health Score ■■■ Number of Dependents-------------✅
Location ■■■ Number of Dependents-----------------✅
Policy Type ■■■ Number of Dependents--------------✅
Previous Claims ■■■ Number of Dependents----------✅
Vehicle Age ■■■ Number of Dependents--------------✅
Credit Score ■■■ Number of Dependents-------------✅
Insurance Duration ■■■ Number of Dependents-------✅
Policy Start Date ■■■ Number of Dependents--------✅
Customer Feedback ■■■ Number of Dependents--------✅
Smoking Status ■■■ Number of Dependents-----------✅
Exercise Frequency ■■■ Number of Dependents-------✅
Property Type ■■■ Number of Dependents------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
4,Number of Dependents,Number of Dependents,Chi-Square,1.536016e+06,0.000000e+00
7,Health Score,Number of Dependents,Kruskal-Wallis,2.437224e+05,3.121024e-46
2,Annual Income,Number of Dependents,Kruskal-Wallis,6.796929e+04,8.741603e-14
12,Credit Score,Number of Dependents,Kruskal-Wallis,7.358708e+02,1.508869e-07
10,Previous Claims,Number of Dependents,Chi-Square,8.562323e+01,8.786966e-07
19,Premium Amount,Number of Dependents,Kruskal-Wallis,4.869565e+03,7.155881e-04
8,Location,Number of Dependents,Chi-Square,2.187406e+01,5.154582e-03
13,Insurance Duration,Number of Dependents,Chi-Square,5.111309e+01,1.736365e-02
11,Vehicle Age,Number of Dependents,Chi-Square,1.021470e+02,2.442912e-02
17,Exercise Frequency,Number of Dependents,Chi-Square,1.839305e+01,1.042671e-01


In [75]:
give_stats_analysis(new_train, "Customer Feedback")

Age ■■■ Customer Feedback-------------------------✅
Gender ■■■ Customer Feedback----------------------✅
Annual Income ■■■ Customer Feedback---------------✅
Marital Status ■■■ Customer Feedback--------------✅
Number of Dependents ■■■ Customer Feedback--------✅
Education Level ■■■ Customer Feedback-------------✅
Occupation ■■■ Customer Feedback------------------✅
Health Score ■■■ Customer Feedback----------------✅
Location ■■■ Customer Feedback--------------------✅
Policy Type ■■■ Customer Feedback-----------------✅
Previous Claims ■■■ Customer Feedback-------------✅
Vehicle Age ■■■ Customer Feedback-----------------✅
Credit Score ■■■ Customer Feedback----------------✅
Insurance Duration ■■■ Customer Feedback----------✅
Policy Start Date ■■■ Customer Feedback-----------✅
Customer Feedback ■■■ Customer Feedback-----------✅
Smoking Status ■■■ Customer Feedback--------------✅
Exercise Frequency ■■■ Customer Feedback----------✅
Property Type ■■■ Customer Feedback---------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
15,Customer Feedback,Customer Feedback,Chi-Square,768008.000000,0.000000e+00
7,Health Score,Customer Feedback,Kruskal-Wallis,243930.576000,4.340066e-48
2,Annual Income,Customer Feedback,Kruskal-Wallis,68342.308900,2.883819e-17
19,Premium Amount,Customer Feedback,Kruskal-Wallis,5059.983182,1.960687e-07
12,Credit Score,Customer Feedback,Kruskal-Wallis,649.542200,1.948388e-03
17,Exercise Frequency,Customer Feedback,Chi-Square,13.182705,4.022461e-02
10,Previous Claims,Customer Feedback,Chi-Square,19.074299,2.648291e-01
18,Property Type,Customer Feedback,Chi-Square,4.626777,3.277785e-01
3,Marital Status,Customer Feedback,Chi-Square,3.249421,5.169858e-01
11,Vehicle Age,Customer Feedback,Chi-Square,36.160320,5.547427e-01


In [76]:
give_stats_analysis(new_train, "Health Score")

Age ■■■ Health Score------------------------------✅
Gender ■■■ Health Score---------------------------✅
Annual Income ■■■ Health Score--------------------✅
Marital Status ■■■ Health Score-------------------✅
Number of Dependents ■■■ Health Score-------------✅
Education Level ■■■ Health Score------------------✅
Occupation ■■■ Health Score-----------------------✅
Health Score ■■■ Health Score---------------------✅
Location ■■■ Health Score-------------------------✅
Policy Type ■■■ Health Score----------------------✅
Previous Claims ■■■ Health Score------------------✅
Vehicle Age ■■■ Health Score----------------------✅
Credit Score ■■■ Health Score---------------------✅
Insurance Duration ■■■ Health Score---------------✅
Policy Start Date ■■■ Health Score----------------✅
Customer Feedback ■■■ Health Score----------------✅
Smoking Status ■■■ Health Score-------------------✅
Exercise Frequency ■■■ Health Score---------------✅
Property Type ■■■ Health Score--------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
7,Health Score,Health Score,SpearmanR,1.000000,0.000000e+00
19,Premium Amount,Health Score,SpearmanR,0.013625,3.073783e-17
2,Annual Income,Health Score,SpearmanR,0.012908,1.251570e-15
0,Age,Health Score,Kruskal-Wallis,96.907798,1.714246e-05
12,Credit Score,Health Score,SpearmanR,0.005636,4.782809e-04
1,Gender,Health Score,Kruskal-Wallis,10.549819,1.161994e-03
4,Number of Dependents,Health Score,Kruskal-Wallis,14.245450,6.551513e-03
10,Previous Claims,Health Score,Kruskal-Wallis,20.849937,7.556923e-03
8,Location,Health Score,Kruskal-Wallis,9.640014,8.066732e-03
13,Insurance Duration,Health Score,Kruskal-Wallis,18.265104,1.932445e-02


In [77]:
give_stats_analysis(new_train, "Annual Income")

Age ■■■ Annual Income-----------------------------✅
Gender ■■■ Annual Income--------------------------✅
Annual Income ■■■ Annual Income-------------------✅
Marital Status ■■■ Annual Income------------------✅
Number of Dependents ■■■ Annual Income------------✅
Education Level ■■■ Annual Income-----------------✅
Occupation ■■■ Annual Income----------------------✅
Health Score ■■■ Annual Income--------------------✅
Location ■■■ Annual Income------------------------✅
Policy Type ■■■ Annual Income---------------------✅
Previous Claims ■■■ Annual Income-----------------✅
Vehicle Age ■■■ Annual Income---------------------✅
Credit Score ■■■ Annual Income--------------------✅
Insurance Duration ■■■ Annual Income--------------✅
Policy Start Date ■■■ Annual Income---------------✅
Customer Feedback ■■■ Annual Income---------------✅
Smoking Status ■■■ Annual Income------------------✅
Exercise Frequency ■■■ Annual Income--------------✅
Property Type ■■■ Annual Income-------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
19,Premium Amount,Annual Income,SpearmanR,-0.063926,0.000000e+00
12,Credit Score,Annual Income,SpearmanR,-0.150058,0.000000e+00
2,Annual Income,Annual Income,SpearmanR,1.000000,0.000000e+00
10,Previous Claims,Annual Income,Kruskal-Wallis,364.472363,7.357893e-74
7,Health Score,Annual Income,SpearmanR,0.012908,1.251570e-15
9,Policy Type,Annual Income,Kruskal-Wallis,7.687293,2.141536e-02
0,Age,Annual Income,Kruskal-Wallis,64.424682,3.761662e-02
8,Location,Annual Income,Kruskal-Wallis,4.414202,1.100191e-01
5,Education Level,Annual Income,Kruskal-Wallis,3.584371,3.099832e-01
3,Marital Status,Annual Income,Kruskal-Wallis,1.671588,4.335301e-01


In [68]:
give_stats_analysis(new_train, "Age")

Age ■■■ Age---------------------------------------✅
Gender ■■■ Age------------------------------------✅
Annual Income ■■■ Age-----------------------------✅
Marital Status ■■■ Age----------------------------✅
Number of Dependents ■■■ Age----------------------✅
Education Level ■■■ Age---------------------------✅
Occupation ■■■ Age--------------------------------✅
Health Score ■■■ Age------------------------------✅
Location ■■■ Age----------------------------------✅
Policy Type ■■■ Age-------------------------------✅
Previous Claims ■■■ Age---------------------------✅
Vehicle Age ■■■ Age-------------------------------✅
Credit Score ■■■ Age------------------------------✅
Insurance Duration ■■■ Age------------------------✅
Policy Start Date ■■■ Age-------------------------✅
Customer Feedback ■■■ Age-------------------------✅
Smoking Status ■■■ Age----------------------------✅
Exercise Frequency ■■■ Age------------------------✅
Property Type ■■■ Age-----------------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
0,Age,Age,Chi-Square,1.766418e+07,0.000000e+00
7,Health Score,Age,Kruskal-Wallis,2.455203e+05,1.763628e-63
2,Annual Income,Age,Kruskal-Wallis,6.847050e+04,1.469649e-18
19,Premium Amount,Age,Kruskal-Wallis,4.945340e+03,4.007837e-05
12,Credit Score,Age,Kruskal-Wallis,6.165932e+02,2.376122e-02
1,Gender,Age,Chi-Square,6.459606e+01,3.646024e-02
8,Location,Age,Chi-Square,1.132974e+02,6.535348e-02
6,Occupation,Age,Chi-Square,1.112562e+02,8.387287e-02
13,Insurance Duration,Age,Chi-Square,4.014972e+02,1.105657e-01
9,Policy Type,Age,Chi-Square,1.027656e+02,2.080060e-01


In [78]:
give_stats_analysis(new_train, "Marital Status")

Age ■■■ Marital Status----------------------------✅
Gender ■■■ Marital Status-------------------------✅
Annual Income ■■■ Marital Status------------------✅
Marital Status ■■■ Marital Status-----------------✅
Number of Dependents ■■■ Marital Status-----------✅
Education Level ■■■ Marital Status----------------✅
Occupation ■■■ Marital Status---------------------✅
Health Score ■■■ Marital Status-------------------✅
Location ■■■ Marital Status-----------------------✅
Policy Type ■■■ Marital Status--------------------✅
Previous Claims ■■■ Marital Status----------------✅
Vehicle Age ■■■ Marital Status--------------------✅
Credit Score ■■■ Marital Status-------------------✅
Insurance Duration ■■■ Marital Status-------------✅
Policy Start Date ■■■ Marital Status--------------✅
Customer Feedback ■■■ Marital Status--------------✅
Smoking Status ■■■ Marital Status-----------------✅
Exercise Frequency ■■■ Marital Status-------------✅
Property Type ■■■ Marital Status------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
3,Marital Status,Marital Status,Chi-Square,768008.000000,0.000000e+00
7,Health Score,Marital Status,Kruskal-Wallis,247196.070010,5.319395e-82
2,Annual Income,Marital Status,Kruskal-Wallis,68467.847519,1.564737e-18
19,Premium Amount,Marital Status,Kruskal-Wallis,5061.298361,1.832459e-07
12,Credit Score,Marital Status,Kruskal-Wallis,679.213771,1.185972e-04
6,Occupation,Marital Status,Chi-Square,21.387437,2.652954e-04
1,Gender,Marital Status,Chi-Square,9.727016,7.723345e-03
18,Property Type,Marital Status,Chi-Square,11.398212,2.243502e-02
11,Vehicle Age,Marital Status,Chi-Square,50.691555,8.164362e-02
13,Insurance Duration,Marital Status,Chi-Square,21.198020,1.710239e-01


In [79]:
give_stats_analysis(new_train, "Vehicle Age")

Age ■■■ Vehicle Age-------------------------------✅
Gender ■■■ Vehicle Age----------------------------✅
Annual Income ■■■ Vehicle Age---------------------✅
Marital Status ■■■ Vehicle Age--------------------✅
Number of Dependents ■■■ Vehicle Age--------------✅
Education Level ■■■ Vehicle Age-------------------✅
Occupation ■■■ Vehicle Age------------------------✅
Health Score ■■■ Vehicle Age----------------------✅
Location ■■■ Vehicle Age--------------------------✅
Policy Type ■■■ Vehicle Age-----------------------✅
Previous Claims ■■■ Vehicle Age-------------------✅
Vehicle Age ■■■ Vehicle Age-----------------------✅
Credit Score ■■■ Vehicle Age----------------------✅
Insurance Duration ■■■ Vehicle Age----------------✅
Policy Start Date ■■■ Vehicle Age-----------------✅
Customer Feedback ■■■ Vehicle Age-----------------✅
Smoking Status ■■■ Vehicle Age--------------------✅
Exercise Frequency ■■■ Vehicle Age----------------✅
Property Type ■■■ Vehicle Age---------------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
11,Vehicle Age,Vehicle Age,Chi-Square,7.296076e+06,0.000000e+00
7,Health Score,Vehicle Age,Kruskal-Wallis,2.412053e+05,1.037434e-26
2,Annual Income,Vehicle Age,Kruskal-Wallis,6.756105e+04,1.867772e-10
13,Insurance Duration,Vehicle Age,Chi-Square,2.045586e+02,2.881638e-03
5,Education Level,Vehicle Age,Chi-Square,9.010998e+01,3.398447e-03
4,Number of Dependents,Vehicle Age,Chi-Square,1.021470e+02,2.442912e-02
16,Smoking Status,Vehicle Age,Chi-Square,3.087366e+01,4.168009e-02
10,Previous Claims,Vehicle Age,Chi-Square,1.791785e+02,6.523835e-02
3,Marital Status,Vehicle Age,Chi-Square,5.069155e+01,8.164362e-02
19,Premium Amount,Vehicle Age,Kruskal-Wallis,4.686575e+03,9.163918e-02


In [80]:
give_stats_analysis(new_train, "Insurance Duration")

Age ■■■ Insurance Duration------------------------✅
Gender ■■■ Insurance Duration---------------------✅
Annual Income ■■■ Insurance Duration--------------✅
Marital Status ■■■ Insurance Duration-------------✅
Number of Dependents ■■■ Insurance Duration-------✅
Education Level ■■■ Insurance Duration------------✅
Occupation ■■■ Insurance Duration-----------------✅
Health Score ■■■ Insurance Duration---------------✅
Location ■■■ Insurance Duration-------------------✅
Policy Type ■■■ Insurance Duration----------------✅
Previous Claims ■■■ Insurance Duration------------✅
Vehicle Age ■■■ Insurance Duration----------------✅
Credit Score ■■■ Insurance Duration---------------✅
Insurance Duration ■■■ Insurance Duration---------✅
Policy Start Date ■■■ Insurance Duration----------✅
Customer Feedback ■■■ Insurance Duration----------✅
Smoking Status ■■■ Insurance Duration-------------✅
Exercise Frequency ■■■ Insurance Duration---------✅
Property Type ■■■ Insurance Duration--------------✅
Premium Amou

,Feature,Target,Statistic Test,Test Statistic,P-Value
13,Insurance Duration,Insurance Duration,Chi-Square,3.072032e+06,0.000000e+00
7,Health Score,Insurance Duration,Kruskal-Wallis,2.415195e+05,7.531030e-29
2,Annual Income,Insurance Duration,Kruskal-Wallis,6.817187e+04,1.266585e-15
19,Premium Amount,Insurance Duration,Kruskal-Wallis,4.880539e+03,4.865332e-04
11,Vehicle Age,Insurance Duration,Chi-Square,2.045586e+02,2.881638e-03
8,Location,Insurance Duration,Chi-Square,3.248350e+01,8.645333e-03
5,Education Level,Insurance Duration,Chi-Square,4.329013e+01,9.215025e-03
4,Number of Dependents,Insurance Duration,Chi-Square,5.111309e+01,1.736365e-02
12,Credit Score,Insurance Duration,Kruskal-Wallis,6.011048e+02,6.114237e-02
17,Exercise Frequency,Insurance Duration,Chi-Square,3.500783e+01,6.828479e-02
